In [2]:
from IPython.display import Image, display
from typing import Annotated
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from PDF_Tool import pdf_tool
from CSV_Tool import csv_tool
from prompt import prompt
# add memory to remeber the previos conversation
memory = MemorySaver()
# Class to handle the updated messages without overwriting


class State(TypedDict):
    messages: Annotated[list, add_messages]

# Create state graph (Workflow) and make the input is the state
graph_builder = StateGraph(State)
# Create tools
tools = [csv_tool,pdf_tool]
# Loading model
llm = ChatOllama(model="mistral",temperature=0)
# Nake chain between llm and tools
llm_with_tools = llm.bind_tools(tools)
tool_node = ToolNode(tools=tools)
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
# add chatbot to graph
graph_builder.add_node("chatbot", chatbot)
# add tool to graph
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "1"}}

user_input = "what is certifacate that bandar had, and what is degree?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what is certifacate that bandar had, and what is degree?
================================== Ai Message ==================================
Tool Calls:
  Bandar Information (b28daf16-53fb-416d-9fb4-18cb44aca4f2)
 Call ID: b28daf16-53fb-416d-9fb4-18cb44aca4f2
  Args:
    __arg1: certificate
  Bandar Information (44b15a0f-dad3-4083-86ff-8d015db236d4)
 Call ID: 44b15a0f-dad3-4083-86ff-8d015db236d4
  Args:
    __arg1: degree
================================= Tool Message =================================
Name: Bandar Information

[Document(id='dce6b771-638e-439e-b797-a73a52f0e61c', metadata={'source': './data/Bandar.pdf', 'file_path': './data/Bandar.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': 'Bander Almutairi', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word for Microsoft 365', 'producer': 'Microsoft® Word for Microsoft 365', 'creationDate': "D:20250106081411+03'00

In [3]:
from IPython.display import Image, display
from typing import Annotated
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from PDF_Tool import pdf_tool
from CSV_Tool import csv_tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Add memory to remember the previous conversation
memory = MemorySaver()

# Class to handle the updated messages without overwriting
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Create state graph (Workflow) and make the input the state
graph_builder = StateGraph(State)

# Create tools
tools = [csv_tool, pdf_tool]

# Loading model
llm = ChatOllama(model="mistral", temperature=0)

# Define the prompt template
template = """
You are a helpful assistant. Answer the following question as accurately and completely as possible.
If the question relates to tools, use the appropriate tool to provide the best response.

User's question: {question}
"""

# Create the prompt template
prompt_template = PromptTemplate(
    input_variables=["question"],
    template=template,
)

# Chain the prompt with the LLM and tools
llm_chain = LLMChain(
    llm=llm,  # This is your ChatOllama model
    prompt=prompt_template
)

# Bind the tools to the LLMChain
llm_with_tools = llm_chain.bind_tools(tools)

# Define the chatbot function
def chatbot(state: State):
    # Format the user input into the prompt
    user_input = state["messages"][-1][1]  # Extract the latest user input
    response = llm_with_tools.run({"question": user_input})
    return {"messages": [response]}

# Add chatbot to graph
graph_builder.add_node("chatbot", chatbot)

# Add tool to graph
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

# Define conditions and edges
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

# Compile the graph with memory
graph = graph_builder.compile(checkpointer=memory)

# Config for the workflow
config = {"configurable": {"thread_id": "1"}}

# User input to test the chatbot
user_input = "what is certificate that Bandar had, and what is degree?"

# Execute the workflow
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

# Display the output
for event in events:
    event["messages"][-1].pretty_print()


C:\Users\B.Almutairi\AppData\Local\Temp\ipykernel_26100\3207365445.py:46: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


AttributeError: 'LLMChain' object has no attribute 'bind_tools'